# core

> API to get structured output from a string using different LLM providers

In [ ]:
#| default_exp core

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| export

import os
from litellm import completion
from pydantic import BaseModel

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

def structured_output(
    model:str, # Model name, see examples here or LiteLLM docs for complete list
    system_prompt:str, # Instructions for LLM to process the input string
    response_format:BaseModel, # User-defined Pydantic model to define output 
    user_prompt:str, # Input string that will be processed
) -> BaseModel:
    
    """Get structured output from `model` by combining system and user prompts and making the right API call.
    See, [here](https://docs.litellm.ai/docs/completion/json_mode#pass-in-json_schema) for full list of APIs available."""
    
    response = completion(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        response_format=response_format
    )
    return response_format.model_validate_json(response.choices[0].message.content)

You can control the model that is used for the call by simply adjusting the string in the `model` variable.

In [ ]:
model="azure/gpt-4o-2024-08-06" # e.g. openai/gpt-4o-2024-08-06 would use the standard OpenAI
system_prompt = "Extract the event information."

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

In [ ]:
user_prompt = "Alice and Bob are going to Carmen's birthday party on 22nd March 2025"

In [ ]:
#| eval: false
r = structured_output(model=model,
                      system_prompt=system_prompt,
                      response_format=CalendarEvent, #Note this is the class name (without the `()`)
                      user_prompt=user_prompt,
                 )

r.model_dump()

{'name': "Carmen's Birthday Party",
 'date': '2025-03-22',
 'participants': ['Alice', 'Bob']}

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()